## 0. Imports

In [ ]:
import warnings

import numpy as np
from obspy import UTCDateTime
from waveform_collection import gather_waveforms_bulk

from rtm import (calculate_time_buffer, define_grid, get_peak_coordinates,
                 grid_search, plot_record_section, plot_time_slice,
                 process_waveforms)

# Ignore benign Matplotlib backend warning due to fig.show()
warnings.filterwarnings(action='ignore', message='Matplotlib is currently using module')

## 1. Define grid

In [ ]:
LON_0 = -175.393  # [deg] Longitude of grid center
LAT_0 = -20.540  # [deg] Latitude of grid center

X_RADIUS = 15  # [deg] E-W grid radius (half of grid "width")
Y_RADIUS = 10  # [deg] N-S grid radius (half of grid "height")
SPACING = 0.5  # [deg] Grid spacing

grid = define_grid(
    lon_0=LON_0,
    lat_0=LAT_0,
    x_radius=X_RADIUS,
    y_radius=Y_RADIUS,
    spacing=SPACING,
    projected=False,
    plot_preview=False,
)

## 2. Grab the data

In [ ]:
# Start and end of time window containing (suspected) events
STARTTIME = UTCDateTime('2022-01-15T03')
ENDTIME = UTCDateTime('2022-01-15T06')

MAX_RADIUS = 7000  # [km] Radius within which to search for stations

# Bulk waveform gather
st = gather_waveforms_bulk(
    LON_0,
    LAT_0,
    MAX_RADIUS,
    STARTTIME,
    ENDTIME,
    'LD?',
    time_buffer=calculate_time_buffer(grid, MAX_RADIUS),
    remove_response=False,  # Avoid dealing with polynomial response
    iris_only=True,
)

# Remove stations with duplicate locations
for station in np.unique([tr.stats.station for tr in st]):
    for tr in st.select(station=station)[1:]:
        st.remove(tr)

# Remove problematic traces
for station in 'I60H2', 'PMG':
    for tr in st.select(station=station):
        st.remove(tr)

## 3. Process the data

In [ ]:
FREQ_MIN = 0.001  # [Hz] Lower bandpass corner
FREQ_MAX = 0.1  # [Hz] Upper bandpass corner

DECIMATION_RATE = 0.1  # [Hz] New sampling rate to use for decimation
SMOOTH_WIN = 15 * 60  # [s] Smoothing window duration

st_proc = process_waveforms(
    st,
    freqmin=FREQ_MIN,
    freqmax=FREQ_MAX,
    envelope=True,
    smooth_win=SMOOTH_WIN,
    decimation_rate=DECIMATION_RATE,
    normalize=True,
)

## 4. Perform grid search

In [ ]:
STACK_METHOD = 'sum'  # Choose either 'sum', 'product', or 'semblance'
TIME_METHOD = 'celerity'  # Choose either 'celerity' or 'fdtd'
CELERITY = 310  # [m/s]

S = grid_search(
    processed_st=st_proc,
    grid=grid,
    time_method=TIME_METHOD,
    starttime=STARTTIME,
    endtime=ENDTIME,
    stack_method=STACK_METHOD,
    celerity=CELERITY,
)

## 4. Plot

In [ ]:
fig_slice = plot_time_slice(
    S, st_proc, label_stations=True, hires=False, plot_peak=True
)
fig_slice.set_size_inches([9, 12])
fig_slice.show()

time_max, y_max, x_max, _, _ = get_peak_coordinates(S, unproject=S.UTM)

fig = plot_record_section(
    st_proc,
    origin_time=time_max,
    source_location=(y_max, x_max),
    plot_celerity='range',
    label_waveforms=True,
)
